In [ ]:
Daily Challenge

1. Preprocess the Data:


from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive

# Importations nécessaires
import os
import zipfile
import shutil
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator


# 2. Définir les chemins
drive_zip_path = '/content/drive/MyDrive/Colab Notebooks/W5/D5/Dogs vs Cats.zip'  # Chemin dans Drive
extract_path = '/content/cats_and_dogs'
original_train_dir = os.path.join(extract_path, 'train/train')  # Adapté à la structure réelle
processed_train_dir = os.path.join(extract_path, 'processed_train')
validation_dir = os.path.join(extract_path, 'validation')
# 3. Nettoyage initial et création des dossiers
!rm -rf {extract_path}  # Suppression complète avant extraction
os.makedirs(extract_path, exist_ok=True)
# 4. Copie depuis Drive vers Colab
!cp "{drive_zip_path}" "{extract_path}/Dogs_vs_Cats.zip"
# 5. Extraction
with zipfile.ZipFile(os.path.join(extract_path, 'Dogs_vs_Cats.zip'), 'r') as zip_ref:
    zip_ref.extractall(extract_path)
# 6. Vérification de la structure
print("Structure après extraction:")
!find {extract_path} -type d | sort
print("\nContenu du dossier train/train:")
!ls -la {original_train_dir} | head -10
# 7. Fonction pour organiser les fichiers
def organize_images(source_dir, train_dest, val_dest, test_size=0.2):
    """Organise les images en train/validation avec séparation par classe"""
    os.makedirs(train_dest, exist_ok=True)
    os.makedirs(val_dest, exist_ok=True)
    # Créer sous-dossiers cats/dogs
    for folder in [train_dest, val_dest]:
        for class_name in ['cats', 'dogs']:
            os.makedirs(os.path.join(folder, class_name), exist_ok=True)
    # Lister les fichiers images
    all_files = [f for f in os.listdir(source_dir)
                if os.path.isfile(os.path.join(source_dir, f))
                and f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    if not all_files:
        raise ValueError(f"Aucune image trouvée dans {source_dir}")
    # Séparation train/validation
    train_files, val_files = train_test_split(all_files, test_size=test_size, random_state=42)
    # Déplacement des fichiers
    for file_list, dest in [(train_files, train_dest), (val_files, val_dest)]:
        for file in file_list:
            src = os.path.join(source_dir, file)
            class_folder = 'cats' if 'cat' in file.lower() else 'dogs'
            dst = os.path.join(dest, class_folder, file)
            shutil.move(src, dst)
# 8. Organiser les images
organize_images(
    source_dir=original_train_dir,
    train_dest=processed_train_dir,
    val_dest=validation_dir,
    test_size=0.2
)
# 9. Vérification après organisation
print("\nStructure après organisation:")
!find {extract_path} -type d | sort
print("\nContenu de processed_train:")
!ls -la {processed_train_dir} | head -10
print("\nContenu de validation:")
!ls -la {validation_dir} | head -10
# 10. Paramètres des images
IMG_HEIGHT = 150
IMG_WIDTH = 150
BATCH_SIZE = 32
EPOCHS = 15
# 11. Augmentation des données
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
validation_datagen = ImageDataGenerator(rescale=1./255)
# 12. Création des générateurs
train_generator = train_datagen.flow_from_directory(
    processed_train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)
validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)
# 13. Affichage des informations
print("\nClasses d'entraînement:", train_generator.class_indices)
print("Nombre d'images d'entraînement:", train_generator.samples)
print("Nombre d'images de validation:", validation_generator.samples)

Structure après extraction:
/content/cats_and_dogs
/content/cats_and_dogs/test
/content/cats_and_dogs/test/test
/content/cats_and_dogs/train
/content/cats_and_dogs/train/train

Contenu du dossier train/train:
total 609272
drwxr-xr-x 2 root root 778240 Jul 11 14:07 .
drwxr-xr-x 3 root root   4096 Jul 11 14:07 ..
-rw-r--r-- 1 root root  12414 Jul 11 14:07 cat.0.jpg
-rw-r--r-- 1 root root  21944 Jul 11 14:07 cat.10000.jpg
-rw-r--r-- 1 root root  27322 Jul 11 14:07 cat.10001.jpg
-rw-r--r-- 1 root root  25723 Jul 11 14:07 cat.10002.jpg
-rw-r--r-- 1 root root  28035 Jul 11 14:07 cat.10003.jpg
-rw-r--r-- 1 root root  12973 Jul 11 14:07 cat.10004.jpg
-rw-r--r-- 1 root root   8245 Jul 11 14:07 cat.10005.jpg

Structure après organisation:
/content/cats_and_dogs
/content/cats_and_dogs/processed_train
/content/cats_and_dogs/processed_train/cats
/content/cats_and_dogs/processed_train/dogs
/content/cats_and_dogs/test
/content/cats_and_dogs/test/test
/content/cats_and_dogs/train
/content/cats_and_dogs/train/train
/content/cats_and_dogs/validation
/content/cats_and_dogs/validation/cats
/content/cats_and_dogs/validation/dogs

Contenu de processed_train:
total 572
drwxr-xr-x 4 root root   4096 Jul 11 14:07 .
drwxr-xr-x 6 root root   4096 Jul 11 14:07 ..
drwxr-xr-x 2 root root 286720 Jul 11 14:07 cats
drwxr-xr-x 2 root root 282624 Jul 11 14:07 dogs

Contenu de validation:
total 152
drwxr-xr-x 4 root root  4096 Jul 11 14:07 .
drwxr-xr-x 6 root root  4096 Jul 11 14:07 ..
drwxr-xr-x 2 root root 69632 Jul 11 14:07 cats
drwxr-xr-x 2 root root 69632 Jul 11 14:07 dogs
Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.

Classes d'entraînement: {'cats': 0, 'dogs': 1}
Nombre d'images d'entraînement: 20000
Nombre d'images de validation: 5000
2. Build the Model:

Le script ci-dessous est un CNN avec trois couches convolutionnelles, chacune suivie d'un max-pooling, puis une couche de flatten, une couche fully connectée avec Dropout, et enfin la couche de sortie adaptée à la classification binaire. La compilation utilise l'optimiseur Adam et la fonction de perte binaire croisée, ce qui est standard pour ce type de problème.


# Importation des modules nécessaires
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

# Définition des dimensions des images (selon ton script initial)
IMG_HEIGHT = 150
IMG_WIDTH = 150

# 2. Construction du modèle CNN
model = Sequential()

# Première couche convolutionnelle avec ReLU et max-pooling
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Deuxième couche convolutionnelle
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Troisième couche convolutionnelle
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten pour transformer la sortie en vecteur
model.add(Flatten())

# Couche fully connectée avec 512 unités et ReLU
model.add(Dense(512, activation='relu'))

# Dropout pour réduire le surapprentissage
model.add(Dropout(0.5))

# Couche de sortie avec sigmoid pour la classification binaire
model.add(Dense(1, activation='sigmoid'))

# Compilation du modèle avec Adam et binary cross-entropy
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Affichage de la structure du modèle
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
Model: "sequential"
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 36992)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │    18,940,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘
 Total params: 19,034,177 (72.61 MB)
 Trainable params: 19,034,177 (72.61 MB)
 Non-trainable params: 0 (0.00 B)
3. Train the Model:


# Entraîner le modèle pendant 15 époques
history = model.fit(
    train_generator,              # Générateur de données d'entraînement avec augmentation
    steps_per_epoch=train_generator.samples // BATCH_SIZE,  # Nombre de pas par époque
    epochs=EPOCHS,                # Nombre total d'époques d'entraînement
    validation_data=validation_generator,  # Données de validation pour évaluer les performances
    validation_steps=validation_generator.samples // BATCH_SIZE,  # Nombre de pas de validation par époque
    verbose=1                     # Affiche la progression pendant l'entraînement
)

# Affichage des performances d'entraînement
import matplotlib.pyplot as plt

# Fonctions auxiliaires pour visualiser l'exactitude et la perte
def plot_training_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs_range = range(EPOCHS)

    # Tracer la courbe d'exactitude
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Exactitude entraînement')
    plt.plot(epochs_range, val_acc, label='Exactitude validation')
    plt.legend(loc='lower right')
    plt.title('Exactitude au fil des époques')

    # Tracer la courbe de perte
    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Perte entraînement')
    plt.plot(epochs_range, val_loss, label='Perte validation')
    plt.legend(loc='upper right')
    plt.title('Perte au fil des époques')

    plt.tight_layout()
    plt.show()

# Visualiser les courbes
plot_training_history(history)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
Epoch 1/15
625/625 ━━━━━━━━━━━━━━━━━━━━ 148s 226ms/step - accuracy: 0.5524 - loss: 0.6840 - val_accuracy: 0.6186 - val_loss: 0.6358
Epoch 2/15
625/625 ━━━━━━━━━━━━━━━━━━━━ 193s 220ms/step - accuracy: 0.6133 - loss: 0.6497 - val_accuracy: 0.7194 - val_loss: 0.5681
Epoch 3/15
625/625 ━━━━━━━━━━━━━━━━━━━━ 141s 218ms/step - accuracy: 0.6756 - loss: 0.5992 - val_accuracy: 0.7286 - val_loss: 0.5331
Epoch 4/15
625/625 ━━━━━━━━━━━━━━━━━━━━ 138s 221ms/step - accuracy: 0.7069 - loss: 0.5698 - val_accuracy: 0.7432 - val_loss: 0.5107
Epoch 5/15
625/625 ━━━━━━━━━━━━━━━━━━━━ 138s 220ms/step - accuracy: 0.7130 - loss: 0.5575 - val_accuracy: 0.7628 - val_loss: 0.4931
Epoch 6/15
625/625 ━━━━━━━━━━━━━━━━━━━━ 138s 221ms/step - accuracy: 0.7250 - loss: 0.5426 - val_accuracy: 0.7778 - val_loss: 0.4704
Epoch 7/15
625/625 ━━━━━━━━━━━━━━━━━━━━ 140s 218ms/step - accuracy: 0.7382 - loss: 0.5254 - val_accuracy: 0.7770 - val_loss: 0.4733
Epoch 8/15
625/625 ━━━━━━━━━━━━━━━━━━━━ 137s 219ms/step - accuracy: 0.7458 - loss: 0.5059 - val_accuracy: 0.8019 - val_loss: 0.4321
Epoch 9/15
625/625 ━━━━━━━━━━━━━━━━━━━━ 141s 218ms/step - accuracy: 0.7621 - loss: 0.4911 - val_accuracy: 0.7792 - val_loss: 0.4594
Epoch 10/15
625/625 ━━━━━━━━━━━━━━━━━━━━ 142s 218ms/step - accuracy: 0.7795 - loss: 0.4705 - val_accuracy: 0.8093 - val_loss: 0.4295
Epoch 11/15
625/625 ━━━━━━━━━━━━━━━━━━━━ 136s 217ms/step - accuracy: 0.7875 - loss: 0.4502 - val_accuracy: 0.8161 - val_loss: 0.4060
Epoch 12/15
625/625 ━━━━━━━━━━━━━━━━━━━━ 203s 315ms/step - accuracy: 0.7870 - loss: 0.4508 - val_accuracy: 0.8249 - val_loss: 0.3974
Epoch 13/15
625/625 ━━━━━━━━━━━━━━━━━━━━ 202s 315ms/step - accuracy: 0.7900 - loss: 0.4452 - val_accuracy: 0.8313 - val_loss: 0.3885
Epoch 14/15
625/625 ━━━━━━━━━━━━━━━━━━━━ 136s 218ms/step - accuracy: 0.8021 - loss: 0.4272 - val_accuracy: 0.8267 - val_loss: 0.3861
Epoch 15/15
625/625 ━━━━━━━━━━━━━━━━━━━━ 142s 218ms/step - accuracy: 0.8054 - loss: 0.4235 - val_accuracy: 0.8393 - val_loss: 0.3657

🔍 Lecture des courbes

Exactitude au fil des époques (gauche)
🔵 Entraînement : progression régulière jusqu’à ~0.80, ce qui montre que le modèle apprend de façon constante.

🟠 Validation : démarre autour de 0.60 et atteint ~0.83, parfois plus élevée que l’entraînement — ce qui peut indiquer une bonne généralisation ou des effets de régularisation.

Perte au fil des époques (droite)
🔵 Entraînement : la perte diminue progressivement vers ~0.40, ce qui confirme l’apprentissage.

🟠 Validation : perte en baisse rapide au départ, puis fluctue légèrement, terminant autour de ~0.35 — ce qui est très bon.

🧠 Ce que ça révèle :

image.png

5. Bonus:


# 📂 1. Définir les chemins (à adapter si nécessaire)
train_dir = '/content/cats_and_dogs/processed_train'
validation_dir = '/content/cats_and_dogs/validation'

# ⚙️ 2. Paramètres de traitement
IMG_HEIGHT = 150
IMG_WIDTH = 150
batch_size = 32
EPOCHS = 15

# 🧰 3. Générateur d'images avec augmentation pour l'entraînement
from tensorflow.keras.preprocessing.image import ImageDataGenerator

image_gen_train = ImageDataGenerator(
    rescale=1./255,               # Normalisation des pixels
    rotation_range=45,            # Rotation aléatoire jusqu’à 45°
    width_shift_range=0.15,       # Décalage horizontal
    height_shift_range=0.15,      # Décalage vertical
    horizontal_flip=True,         # Flip horizontal
    zoom_range=0.5                # Zoom jusqu’à 50%
)

# 📡 4. Générateur d'entraînement
train_data_gen = image_gen_train.flow_from_directory(
    batch_size=batch_size,
    directory=train_dir,
    shuffle=True,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode='binary'
)

# 🧼 5. Générateur de validation (sans augmentation)
image_gen_val = ImageDataGenerator(rescale=1./255)

val_data_gen = image_gen_val.flow_from_directory(
    batch_size=batch_size,
    directory=validation_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode='binary'
)

# 🧠 6. Réentraînement du modèle avec les nouvelles données augmentées
history_augmented = model.fit(
    train_data_gen,
    steps_per_epoch=train_data_gen.samples // batch_size,
    epochs=EPOCHS,
    validation_data=val_data_gen,
    validation_steps=val_data_gen.samples // batch_size,
    verbose=1
)

# 📈 7. Visualisation des performances
import matplotlib.pyplot as plt

def plot_augmented_training(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs_range = range(len(acc))

    plt.figure(figsize=(12, 5))

    # Courbes d'exactitude
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Exactitude entraînement')
    plt.plot(epochs_range, val_acc, label='Exactitude validation')
    plt.title("Exactitude avec augmentation")
    plt.legend(loc='lower right')

    # Courbes de perte
    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Perte entraînement')
    plt.plot(epochs_range, val_loss, label='Perte validation')
    plt.title("Perte avec augmentation")
    plt.legend(loc='upper right')

    plt.tight_layout()
    plt.show()

# 🔍 8. Affichage
plot_augmented_training(history_augmented)


Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
Epoch 1/15
625/625 ━━━━━━━━━━━━━━━━━━━━ 1221s 2s/step - accuracy: 0.5220 - loss: 0.7191 - val_accuracy: 0.5727 - val_loss: 0.6753
Epoch 2/15
625/625 ━━━━━━━━━━━━━━━━━━━━ 1223s 2s/step - accuracy: 0.5661 - loss: 0.6804 - val_accuracy: 0.6226 - val_loss: 0.6515
Epoch 3/15
625/625 ━━━━━━━━━━━━━━━━━━━━ 1221s 2s/step - accuracy: 0.6134 - loss: 0.6569 - val_accuracy: 0.6841 - val_loss: 0.5932
Epoch 4/15
625/625 ━━━━━━━━━━━━━━━━━━━━ 1221s 2s/step - accuracy: 0.6570 - loss: 0.6153 - val_accuracy: 0.7388 - val_loss: 0.5344
Epoch 5/15
625/625 ━━━━━━━━━━━━━━━━━━━━ 1194s 2s/step - accuracy: 0.6872 - loss: 0.5804 - val_accuracy: 0.7612 - val_loss: 0.5084
Epoch 6/15
625/625 ━━━━━━━━━━━━━━━━━━━━ 1216s 2s/step - accuracy: 0.7054 - loss: 0.5712 - val_accuracy: 0.7716 - val_loss: 0.4868
Epoch 7/15
625/625 ━━━━━━━━━━━━━━━━━━━━ 1282s 2s/step - accuracy: 0.7192 - loss: 0.5516 - val_accuracy: 0.7628 - val_loss: 0.4979
Epoch 8/15
625/625 ━━━━━━━━━━━━━━━━━━━━ 1176s 2s/step - accuracy: 0.7203 - loss: 0.5431 - val_accuracy: 0.7684 - val_loss: 0.4809
Epoch 9/15
625/625 ━━━━━━━━━━━━━━━━━━━━ 1222s 2s/step - accuracy: 0.7265 - loss: 0.5392 - val_accuracy: 0.7726 - val_loss: 0.4817
Epoch 10/15
625/625 ━━━━━━━━━━━━━━━━━━━━ 1282s 2s/step - accuracy: 0.7370 - loss: 0.5282 - val_accuracy: 0.7945 - val_loss: 0.4349
Epoch 11/15
625/625 ━━━━━━━━━━━━━━━━━━━━ 1284s 2s/step - accuracy: 0.7491 - loss: 0.5072 - val_accuracy: 0.7879 - val_loss: 0.4510
Epoch 12/15
625/625 ━━━━━━━━━━━━━━━━━━━━ 1219s 2s/step - accuracy: 0.7581 - loss: 0.5003 - val_accuracy: 0.7961 - val_loss: 0.4381
Epoch 13/15
625/625 ━━━━━━━━━━━━━━━━━━━━ 1223s 2s/step - accuracy: 0.7642 - loss: 0.4929 - val_accuracy: 0.8005 - val_loss: 0.4241
Epoch 14/15
625/625 ━━━━━━━━━━━━━━━━━━━━ 1223s 2s/step - accuracy: 0.7669 - loss: 0.4841 - val_accuracy: 0.8167 - val_loss: 0.4115
Epoch 15/15
625/625 ━━━━━━━━━━━━━━━━━━━━ 1222s 2s/step - accuracy: 0.7658 - loss: 0.4799 - val_accuracy: 0.7744 - val_loss: 0.4751

Interprétation :

📈 Courbe d'exactitude (gauche) Exactitude d’entraînement (bleue) : elle augmente progressivement, atteignant environ 0.78, ce qui montre que le modèle apprend correctement.

Exactitude de validation (orange) : suit une trajectoire similaire et atteint environ 0.80, souvent légèrement supérieure à celle de l’entraînement — ce qui est un très bon signe.

✅ Conclusion : il n'y a pas de surapprentissage apparent, les deux courbes évoluent ensemble de façon fluide.

📉 Courbe de perte (droite) Perte d’entraînement (bleue) : diminue régulièrement vers ~0.45, ce qui indique une amélioration du modèle.

Perte de validation (orange) : commence plus haut (~0.60), puis diminue en parallèle, atteignant ~0.42, meilleure que la perte d’entraînement à la fin.

✅ Conclusion : la validation confirme que le modèle se généralise bien, probablement grâce à l’effet de régularisation introduit par l’augmentation.

image.png

📊 Globalement, le modèle semble plus performant et plus stable que lors du premier entraînement sans augmentation.

⚔️ Comparaison directe entre les deux entraînements

image.png image.png

💡 Ce qu’on peut conclure Le modèle sans augmentation atteint un meilleur score brut de validation, mais montre des signes de surapprentissage léger : fluctuation de la perte et exactitude plus haute que l’entraînement.

Le modèle avec augmentation est plus régulier, robuste et généralise mieux — même si les scores ne sont pas aussi “hauts”, leur stabilité est bien plus rassurante pour des cas réels.

En résumé :

🎯 L’augmentation de données agit comme un bouclier contre la suradaptation. Elle rend le modèle moins spectaculaire dans les scores d’entraînement, mais beaucoup plus fiable dans la durée.